In [33]:
import re

class MiniLangScanner:
    
    def __init__(self, file_path):
        
        #Read Source Code from the File
        with open(file_path, 'r', encoding='utf-8') as file:
            self.source_code = file.read()
            
        #Creating List to store the Tokens   
        self.tokens = []
        
        #Set of Minilang Keywords as given in Task
        self.keywords = {'if', 'else', 'print', 'true', 'false'}
        self.current_position = 0

    def scan(self):
        
        #Tokenize the Minilang Scource Code
        while self.current_position < len(self.source_code):
       
            #Ignoring the Whitespaces and Comments
            self._skip_whitespace_and_comments()
            
            #Reach the End of the File and Check
            if self.current_position >= len(self.source_code):
                break
                
            #Matching the types of Tokens to respective
            if self._match_identifier():
                continue
            elif self._match_literal():
                continue
            elif self._match_operator():
                continue
                
            #Condition for invalid Token    
            else:
                self._report_error("Invalid token")
                
        return self.tokens

    #Function for skipping white spaces and comment lines with // 
    def _skip_whitespace_and_comments(self):
        
        whitespace_and_comment_pattern = re.compile(r'\s+|//.*')
        match = whitespace_and_comment_pattern.match(self.source_code, self.current_position)
        
        while match:
            self.current_position = match.end()
            match = whitespace_and_comment_pattern.match(self.source_code, self.current_position)

    #Function for Matching the Identifiers
    def _match_identifier(self):
        
        identifier_pattern = re.compile(r'[a-zA-Z][a-zA-Z0-9]*')
        match = identifier_pattern.match(self.source_code, self.current_position)
        
        #If the string matched get it & Append for keywords and identifiers
        if match:
            lexeme = match.group()
            if lexeme in self.keywords:
                self.tokens.append(("Keyword", lexeme))
            else:
                self.tokens.append(("Identifier", lexeme))
            self.current_position = match.end()
            return True
        
        return False
    
    #Function for matching Literals
    def _match_literal(self):
        
        literal_pattern = re.compile(r'\d+|true|false|"([^"]*)"')
        match = literal_pattern.match(self.source_code, self.current_position)
        
        #If matched get the string and Append for Literals of Int, Bool, String
        if match:
            lexeme = match.group()
            if lexeme.isdigit():
                self.tokens.append(("Integer Literal", int(lexeme)))
            elif lexeme == 'true' or lexeme == 'false':
                self.tokens.append(("Boolean Literal", lexeme == 'true'))
            elif lexeme.startswith('"') and lexeme.endswith('"'):
                self.tokens.append(("String Literal", lexeme[1:-1]))
            self.current_position = match.end()
            return True
        
        return False

    #Fucntion for Matching the Operators asked for in assignment (+, -, *, /, =, ==, != )
    def _match_operator(self):
        
        operator_pattern = re.compile(r'==|!=|[=+\-*/]')
        match = operator_pattern.match(self.source_code, self.current_position)
        
        #If Matched get the string and append for Operator
        if match:
            lexeme = match.group()
            self.tokens.append(("Operator", lexeme))
            self.current_position = match.end()
            return True
        
        return False

    #Fucntion for handling Lexical Errors
    def _report_error(self, message):
        
        error_position = min(self.current_position, len(self.source_code))
        snippet = self.source_code[max(0, error_position - 10):min(len(self.source_code), error_position + 10)]
        raise ValueError(f"Lexical Error: {message} at position {error_position}\n{snippet}")


# Parser Implementation
class MiniLangParser:
    
    def __init__(self, tokens):
        
         # Initialize MiniLangParser with tokens and current token index
        self.tokens = tokens
        self.current_token_index = 0

    def parse(self):
        return self.parse_program()

     # Grammar Rules
    def parse_program(self):
        
        statements = []
        while self.current_token_index < len(self.tokens):
            statement = self.parse_statement()
            statements.append(statement)
        return statements

    # Parse a MiniLang program composed of statements
    def parse_statement(self):
        
        token_type, lexeme = self.tokens[self.current_token_index]

        # Parse if statements
        if token_type == "Keyword" and lexeme == "if":
            return self.parse_if_statement()
        
        # Parse print statements
        elif token_type == "Keyword" and lexeme == "print":
            return self.parse_print_statement()
        
        # Parse else statements
        elif token_type == "Keyword" and lexeme == "else":
            return self.parse_else_statement()
        
        # Parse assignment statements
        elif token_type == "Identifier":
            return self.parse_assignment_statement()
        
        # Raise an error for unexpected tokens
        else:
            self.raise_parser_error(f"Unexpected token: {token_type} with lexeme: {lexeme}")

            
    # Parse else statements
    def parse_else_statement(self):
        
        self.match_token("Keyword")  
        else_block = self.parse_block()
        return {"type": "else_statement", "else_block": else_block}

    
    # Parse print statements
    def parse_print_statement(self):
        
        self.match_token("Keyword")
        expression = self.parse_expression() 
        return {"type": "print_statement", "expression": expression}
    
    
    # Parse assignment statement
    def parse_assignment_statement(self):
        
        identifier = self.tokens[self.current_token_index][1]
        self.match_token("Identifier")
        self.match_token("Operator")  
        expression = self.parse_expression()
        return {"type": "assignment_statement", "identifier": identifier, "expression": expression}

    # Task: Parse if statement, including condition, if block, and optional else block
    def parse_if_statement(self):
        
        self.match_token("Keyword")
        condition = self.parse_expression()  
        if_block = self.parse_block()
        else_block = None

        if self.current_token_index < len(self.tokens) and self.tokens[self.current_token_index][1] == "else":
            self.match_token("Keyword")
            else_block = self.parse_block()

        return {"type": "if_statement", "condition": condition, "if_block": if_block, "else_block": else_block}

    
    def parse_expression(self):
        return self.parse_logical_expression()

        
    # Parse logical expressions, including logical operators
    def parse_logical_expression(self):
        
        left_operand = self.parse_arithmetic_expression()
        while self.current_token_index < len(self.tokens):
            operator = self.tokens[self.current_token_index][1]
            if operator in {"==", "!="}:
                self.match_token("Operator")
                right_operand = self.parse_arithmetic_expression()
                left_operand = {"type": "logical_expression", "operator": operator, "left_operand": left_operand, "right_operand": right_operand}
            else:
                break
        return left_operand

    
    # Parse arithmetic expressions, including arithmetic operators
    def parse_arithmetic_expression(self):
        
        return self.parse_term()

    # Parse term, including term operators
    def parse_term(self):
        
        left_operand = self.parse_factor()
        while self.current_token_index < len(self.tokens):
            operator = self.tokens[self.current_token_index][1]
            if operator in {"+", "-"}:
                self.match_token("Operator")
                right_operand = self.parse_factor()
                left_operand = {"type": "arithmetic_expression", "operator": operator, "left_operand": left_operand, "right_operand": right_operand}
            else:
                break
        return left_operand

    
    # Parse factors, including literals and identifiers
    def parse_factor(self):
        
        token_type, lexeme = self.tokens[self.current_token_index]

        if token_type == "Integer Literal":
            self.match_token("Integer Literal")
            return {"type": "integer_literal", "value": int(lexeme)}
        elif token_type == "Boolean Literal":
            self.match_token("Boolean Literal")
            return {"type": "boolean_literal", "value": lexeme == "true"}
        elif token_type == "String Literal":
            self.match_token("String Literal")
            return {"type": "string_literal", "value": lexeme}
        elif token_type == "Identifier":
            identifier = lexeme
            self.match_token("Identifier")
            return {"type": "identifier", "value": identifier}
        else:
            self.raise_parser_error("Invalid factor")

    # Parse a block of statements
    def parse_block(self):
        
        statements = self.parse_program()
        return {"type": "block", "statements": statements}

    
    # Match the expected token type, else raise a parser error
    def match_token(self, expected_token_type):
        
        token_type, lexeme = self.tokens[self.current_token_index]
        if token_type == expected_token_type:
            self.current_token_index += 1
        else:
            self.raise_parser_error(f"Expected {expected_token_type}, but found {token_type}")

            
    # Raise a parser error with a descriptive message
    def raise_parser_error(self, message):
        
        raise ValueError(f"Parser Error: {message}")

        
        
    # Print the abstract syntax tree with proper indentation
    def print_tree(self, node, indent=0):
        
        if isinstance(node, dict):
            for key, value in node.items():
                if isinstance(value, list):
                    print("  " * indent + f"{key}: [")
                    for item in value:
                        self.print_tree(item, indent + 2)
                    print("  " * indent + "]")
                elif isinstance(value, dict):
                    print("  " * indent + f"{key}: {{")
                    self.print_tree(value, indent + 2)
                    print("  " * indent + "}")
                else:
                    print("  " * indent + f"{key}: {value}")
        elif isinstance(node, list):
            for item in node:
                self.print_tree(item, indent)
                
                
def test_minilang(file_path):
    
    scanner = MiniLangScanner(file_path)
    tokens = scanner.scan()

    parser = MiniLangParser(tokens)
    abstract_syntax_tree = parser.parse()

    # Print the abstract syntax tree
    print(f"Abstract Syntax Tree for {file_path}:\n\n")
    parser.print_tree(abstract_syntax_tree)
    print("\n" + "=" * 50 + "\n")  # Separation line

    
# Test with different files

#If else case
test_minilang("case1.txt")

#Case for != operator
test_minilang("case2.txt")

#Case for Boolean
test_minilang("case3.txt")

#Edge Case to check an empty file
test_minilang("edge case 1.txt")

#Edge Case to check aa file with white spaces only
test_minilang("edge case 2.txt")

#Edge Case to check a file with comments only
test_minilang("edge case 3.txt")

#Edge Case to check a file with Long Identifiers
test_minilang("edge case 4.txt")


Abstract Syntax Tree for case1.txt:


type: assignment_statement
identifier: x
expression: {
    type: integer_literal
    value: 5
}
type: assignment_statement
identifier: y
expression: {
    type: integer_literal
    value: 10
}
type: if_statement
condition: {
    type: logical_expression
    operator: ==
    left_operand: {
        type: identifier
        value: x
    }
    right_operand: {
        type: identifier
        value: y
    }
}
if_block: {
    type: block
    statements: [
        type: print_statement
        expression: {
            type: string_literal
            value: x is equal to y
        }
        type: else_statement
        else_block: {
            type: block
            statements: [
                type: print_statement
                expression: {
                    type: string_literal
                    value: x is not equal to y
                }
            ]
        }
    ]
}
else_block: None


Abstract Syntax Tree for case2.txt:


type: assign